In [1]:
%matplotlib inline
import datetime as dt
import pyodbc 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# data colelction and preprocessing

In [5]:
#Create a connection from Python to SQL
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=HP;"
                      "Database=AdventureWorks2014;"
                      "Trusted_Connection=yes;")

In [150]:
data = pd.read_sql_query("""
select [CustomerID],
sso.[OrderDate] as  Invoicedate,
ssd.[UnitPrice],
ssd.ProductID as stockcode,
ssc.[Quantity],
sso.PurchaseOrderNumber as invioceNo,
sso.SalesOrderNumber as salesInivoice,
pp.name,
tery.[Name] as country
from [Sales].[SalesOrderHeader] as sso
inner join [Sales].[SalesTerritory] as tery
on tery.[TerritoryID] = sso.[TerritoryID]
inner join [Sales].[SalesOrderDetail] as ssd
on sso.[SalesOrderID] = ssd.[SalesOrderID]
left join [Sales].[ShoppingCartItem] as ssc
on ssc.ProductID = ssd.ProductID
left join [Production].[Product] as pp
on pp.[ProductID] = ssc.ProductID
left join [Production].[ProductModelProductDescriptionCulture] as ppm
on ppm.ProductModelID = pp.ProductModelID
inner join [Production].[ProductDescription] as pdd
on pdd.ProductDescriptionID = ppm.ProductDescriptionID

""", cnxn)


In [151]:
data.head()

,CustomerID,Invoicedate,UnitPrice,stockcode,Quantity,invioceNo,salesInivoice,name,country
0,29825,2012-05-30,22.0342,862,3,PO522154989,SO46604,"Full-Finger Gloves, M",Southeast
1,29825,2012-05-30,22.0342,862,3,PO522154989,SO46604,"Full-Finger Gloves, M",Southeast
2,29825,2012-05-30,22.0342,862,3,PO522154989,SO46604,"Full-Finger Gloves, M",Southeast
3,29825,2012-05-30,22.0342,862,3,PO522154989,SO46604,"Full-Finger Gloves, M",Southeast
4,29825,2012-05-30,22.0342,862,3,PO522154989,SO46604,"Full-Finger Gloves, M",Southeast


In [152]:
data.tail()

,CustomerID,Invoicedate,UnitPrice,stockcode,Quantity,invioceNo,salesInivoice,name,country
10009,11657,2014-06-30,53.99,881,4,None,SO75095,"Short-Sleeve Classic Jersey, S",Southwest
10010,11657,2014-06-30,53.99,881,4,None,SO75095,"Short-Sleeve Classic Jersey, S",Southwest
10011,11657,2014-06-30,53.99,881,4,None,SO75095,"Short-Sleeve Classic Jersey, S",Southwest
10012,11657,2014-06-30,53.99,881,4,None,SO75095,"Short-Sleeve Classic Jersey, S",Southwest
10013,11657,2014-06-30,53.99,881,4,None,SO75095,"Short-Sleeve Classic Jersey, S",Southwest


In [153]:
data['name'] = data['name'].str.strip()
data.dropna(axis=0, subset=['salesInivoice'], inplace=True)
data['salesInivoice'] = data['salesInivoice'].astype('str')
data = data[~data['salesInivoice'].str.contains('C')]

In [154]:
basket = (data[data['country'] =="France"]
          .groupby(['salesInivoice', 'name'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('salesInivoice'))

In [155]:

basket.head()

name,"Full-Finger Gloves, M","Racing Socks, M","Short-Sleeve Classic Jersey, S"
salesInivoice,,,
SO46626,18.0,0.0,0.0
SO46627,18.0,0.0,0.0
SO46941,18.0,0.0,0.0
SO46985,18.0,0.0,0.0
SO46987,18.0,0.0,0.0


In [156]:

# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [146]:

basket_sets = basket.applymap(encode_units)

In [157]:

# Build up the frequent items
frequent_itemsets = apriori(basket_sets, min_support=0.00, use_colnames=True)

In [158]:

frequent_itemsets.head()

,support,itemsets
0,0.209091,"(Full-Finger Gloves, M)"
1,0.327273,"(Racing Socks, M)"
2,0.509091,"(Short-Sleeve Classic Jersey, S)"
3,0.000000,"(Full-Finger Gloves, M, Racing Socks, M)"
4,0.000000,"(Short-Sleeve Classic Jersey, S, Full-Finger G..."


In [159]:

# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.00)
rules

C:\Users\Ajay\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\association_rules.py:105: RuntimeWarning: invalid value encountered in double_scalars
  "confidence": lambda sAC, sA, _: sAC/sA,
C:\Users\Ajay\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\association_rules.py:106: RuntimeWarning: invalid value encountered in double_scalars
  "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC)/sC,


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Full-Finger Gloves, M)","(Racing Socks, M)",0.209091,0.327273,0.000000,0.000000,0.000000,-0.068430,0.672727
1,"(Racing Socks, M)","(Full-Finger Gloves, M)",0.327273,0.209091,0.000000,0.000000,0.000000,-0.068430,0.790909
2,"(Short-Sleeve Classic Jersey, S)","(Full-Finger Gloves, M)",0.509091,0.209091,0.000000,0.000000,0.000000,-0.106446,0.790909
3,"(Full-Finger Gloves, M)","(Short-Sleeve Classic Jersey, S)",0.209091,0.509091,0.000000,0.000000,0.000000,-0.106446,0.490909
4,"(Short-Sleeve Classic Jersey, S)","(Racing Socks, M)",0.509091,0.327273,0.045455,0.089286,0.272817,-0.121157,0.738681
5,"(Racing Socks, M)","(Short-Sleeve Classic Jersey, S)",0.327273,0.509091,0.045455,0.138889,0.272817,-0.121157,0.570088
6,"(Short-Sleeve Classic Jersey, S, Racing Socks, M)","(Full-Finger Gloves, M)",0.045455,0.209091,0.000000,0.000000,0.000000,-0.009504,0.790909
7,"(Full-Finger Gloves, M)","(Short-Sleeve Classic Jersey, S, Racing Socks, M)",0.209091,0.045455,0.000000,0.000000,0.000000,-0.009504,0.954545


From the above table, the high lift value means that the product occurs more frequently than would be expected given the number of transaction and product combinations.

In [161]:
# In this section, I filter the table and look where the  lift  is above (0) and the confidence is baove (.5)
rules[ (rules['lift'] >= 0.00) &
       (rules['confidence'] >= 0.05) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(Short-Sleeve Classic Jersey, S)","(Racing Socks, M)",0.509091,0.327273,0.045455,0.089286,0.272817,-0.121157,0.738681
5,"(Racing Socks, M)","(Short-Sleeve Classic Jersey, S)",0.327273,0.509091,0.045455,0.138889,0.272817,-0.121157,0.570088


In looking at the rules, it appears that Short-Sleeve Classic Jersey and Racing Socks are purchased together. 

The question is that how much would provide the opportunity of buying the Racing Socks would drive sales of Short-Sleeve Classic Jersey or vice versa? 


In [163]:
basket['Short-Sleeve Classic Jersey, S'].sum()


1344.0

In [164]:
basket['Racing Socks, M'].sum()

1512.0

This result implies that, if we sell 1512 of the racing socks, 1344 of Short-Sleeve Classic Jersey may likely buy by the same customer in France through recommendations. 

What is also interesting is to see how the combinations vary by country of purchase. Let’s check out what some popular combinations might be in Australia:

In [170]:
basket2 = (data[data['country'] == "Australia"]
          .groupby(['salesInivoice', 'name'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('salesInivoice'))

In [172]:
rules2[ (rules2['lift'] >= 0.00) &
        (rules2['confidence'] >= 0.05)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [181]:
basket2 = (data[data['country'] == "Canada"]
          .groupby(['salesInivoice', 'name'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('salesInivoice'))

basket_sets2 = basket2.applymap(encode_units)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.00, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=0.00)

rules2[ (rules2['lift'] >= 0.00) &
        (rules2['confidence'] >= 0.05)]

C:\Users\Ajay\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\association_rules.py:105: RuntimeWarning: invalid value encountered in double_scalars
  "confidence": lambda sAC, sA, _: sAC/sA,
C:\Users\Ajay\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\association_rules.py:106: RuntimeWarning: invalid value encountered in double_scalars
  "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC)/sC,


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"(Short-Sleeve Classic Jersey, S)","(Racing Socks, M)",0.375000,0.382812,0.058594,0.156250,0.408163,-0.084961,0.731481
5,"(Racing Socks, M)","(Short-Sleeve Classic Jersey, S)",0.382812,0.375000,0.058594,0.153061,0.408163,-0.084961,0.737952


In [182]:
basket2['Short-Sleeve Classic Jersey, S'].sum()


2304.0

In [183]:
basket2['Racing Socks, M'].sum()

4116.0

This result implies that, if we sell 4116 of the racing socks, 2304 of Short-Sleeve Classic Jersey may likely buy by the same customer in Canada. 